In [58]:
#!pip install yfinance


In [59]:
#!pip install pyportfolioopt


In [399]:
import yfinance as yf
import pandas as pd
import numpy as np
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.hierarchical_portfolio import HRPOpt
from pypfopt.cla import CLA
from pypfopt import risk_models
from pypfopt import expected_returns
import datetime as dt
from dateutil.relativedelta import relativedelta

In [234]:
tickers = ['SPY', 'EEM', 'TIP', 'IAU', 'DBC', 'TLT', 'VGK', 'IWM','LQD','VNQ', 'SHY', 'IWM','QQQ','HYG']

In [235]:
prices = yf.download(tickers=tickers)

[*********************100%%**********************]  13 of 13 completed


In [236]:
prices.to_csv("price_data.csv")

In [237]:
prices

Adj Close                                               \
                  DBC        EEM        HYG        IAU         IWM   
Date                                                                 
1993-01-29        NaN        NaN        NaN        NaN         NaN   
1993-02-01        NaN        NaN        NaN        NaN         NaN   
1993-02-02        NaN        NaN        NaN        NaN         NaN   
1993-02-03        NaN        NaN        NaN        NaN         NaN   
1993-02-04        NaN        NaN        NaN        NaN         NaN   
...               ...        ...        ...        ...         ...   
2023-10-31  24.510000  36.700001  72.181999  37.590000  164.520004   
2023-11-01  24.549999  37.040001  72.870003  37.459999  165.460007   
2023-11-02  24.809999  37.689999  73.690002  37.599998  169.869995   
2023-11-03  24.680000  38.459999  74.410004  37.730000  174.490005   
2023-11-06  24.780001  38.770000  74.110001  37.430000  172.309998   

                                                                       ...  \
                   LQD         QQQ        SHY         SPY         TIP  ...   
Date                                                                   ...   
1993-01-29         NaN         NaN        NaN   24.941385         NaN  ...   
1993-02-01         NaN         NaN        NaN   25.118788         NaN  ...   
1993-02-02         NaN         NaN        NaN   25.172012         NaN  ...   
1993-02-03         NaN         NaN        NaN   25.438101         NaN  ...   
1993-02-04         NaN         NaN        NaN   25.544535         NaN  ...   
...                ...         ...        ...         ...         ...  ...   
2023-10-31   98.806000  350.869995  80.800003  418.200012  102.348000  ...   
2023-11-01  100.220001  356.959991  80.980003  422.660004  103.239998  ...   
2023-11-02  101.410004  363.440002  80.989998  430.760010  103.709999  ...   
2023-11-03  101.910004  367.709991  81.209999  434.690002  104.239998  ...   
2023-11-06  101.370003  369.209991  81.059998  435.690002  103.970001  ...   

                Volume                                                 \
                   IAU         IWM         LQD         QQQ        SHY   
Date                                                                    
1993-01-29         NaN         NaN         NaN         NaN        NaN   
1993-02-01         NaN         NaN         NaN         NaN        NaN   
1993-02-02         NaN         NaN         NaN         NaN        NaN   
1993-02-03         NaN         NaN         NaN         NaN        NaN   
1993-02-04         NaN         NaN         NaN         NaN        NaN   
...                ...         ...         ...         ...        ...   
2023-10-31   9450600.0  31533200.0  28805700.0  47671900.0  3917400.0   
2023-11-01  13901600.0  44367100.0  32111800.0  58992700.0  7362900.0   
2023-11-02   4212100.0  50458100.0  31295500.0  58162400.0  7828400.0   
2023-11-03   8184200.0  57041300.0  38496200.0  53239100.0  8010400.0   
2023-11-06   4401156.0  34647500.0  17799845.0  37562847.0  6061228.0   

                                                                    
                  SPY        TIP         TLT        VGK        VNQ  
Date                                                                
1993-01-29    1003200        NaN         NaN        NaN        NaN  
1993-02-01     480500        NaN         NaN        NaN        NaN  
1993-02-02     201300        NaN         NaN        NaN        NaN  
1993-02-03     529400        NaN         NaN        NaN        NaN  
1993-02-04     531500        NaN         NaN        NaN        NaN  
...               ...        ...         ...        ...        ...  
2023-10-31   79665200  5219300.0  37269200.0  4617900.0  5964700.0  
2023-11-01   98068100  3244500.0  72308500.0  2770500.0  7021600.0  
2023-11-02   94938900  2869700.0  69739400.0  2951500.0  6450300.0  
2023-11-03  100110800  3444000.0  77668500.0  2119000.0  6394100.0  
2023-11-06   67646

In [352]:
tickers = ['SPY', 'EEM', 'TIP', 'IAU', 'DBC', 'TLT', 'VGK', 'IWM','LQD','VNQ', 'SHY','QQQ','HYG']

In [353]:
adjclose = prices[prices.columns[:len(tickers)+1]]
adjclose.columns = [x[1] for x in adjclose.columns]
adjclose = adjclose.reset_index().rename(columns={'Date':'date'})
adjclose['date'] = pd.to_datetime(adjclose['date']).dt.date
adjclose.set_index(['date'], inplace=True)
adjclose = adjclose[tickers]
adjclose.dropna(inplace=True)

In [354]:
for c in adjclose.columns:
    print(c , ": ", adjclose[c].first_valid_index())

SPY :  2007-04-11
EEM :  2007-04-11
TIP :  2007-04-11
IAU :  2007-04-11
DBC :  2007-04-11
DBC :  2007-04-11
TLT :  2007-04-11
VGK :  2007-04-11
IWM :  2007-04-11
LQD :  2007-04-11
VNQ :  2007-04-11
SHY :  2007-04-11
QQQ :  2007-04-11
HYG :  2007-04-11


In [424]:
#adjclose_t = 
rebal_date = dt.datetime.strptime('2012-01-01', '%Y-%m-%d').date()
port_wts = {}
for ddate in adjclose.index.unique():
    if ddate>= rebal_date:
        adjclose_t = adjclose.loc[(adjclose.index <= ddate)]
        mu = expected_returns.mean_historical_return(adjclose_t)
        S = risk_models.exp_cov(adjclose_t, span = 180)
       # ret = adjclose_t.shift(-1)/adjclose_t-1
        ef = EfficientFrontier(mu, S,weight_bounds=(0,0.25))
        weights = ef.efficient_risk(target_volatility=0.1)   
        port_wts[ddate] = weights
        rebal_date = rebal_date + relativedelta(months=1)
    

In [425]:
port_wts

{datetime.date(2012, 1, 3): OrderedDict([('SPY', 8.58811e-11),
              ('EEM', 1.304405e-10),
              ('TIP', 0.2499999994687349),
              ('IAU', 0.2499999999692504),
              ('DBC', 1.371573e-10),
              ('TLT', 0.249999999864619),
              ('VGK', 2.85928e-11),
              ('IWM', 9.25786e-11),
              ('LQD', 0.2342551485304086),
              ('VNQ', 6.72931e-11),
              ('SHY', 2.415574e-10),
              ('QQQ', 0.0157448501974034),
              ('HYG', 1.0083568e-09)]),
 datetime.date(2012, 2, 1): OrderedDict([('SPY', 2.99901e-11),
              ('EEM', 6.72321e-11),
              ('TIP', 0.2499999998302696),
              ('IAU', 0.2499999999866207),
              ('DBC', 4.3788e-11),
              ('TLT', 0.2499999999477376),
              ('VGK', 8.8264e-12),
              ('IWM', 3.91043e-11),
              ('LQD', 5.050136e-10),
              ('VNQ', 2.66093e-11),
              ('SHY', 6.07588e-11),
              ('QQQ',

In [426]:
portfolio_wts = pd.DataFrame(port_wts).T
#asset_returns = 
adjclose_return = adjclose.loc[(adjclose.index.isin(portfolio_wts.index))].shift(-1)/adjclose.loc[(adjclose.index.isin(portfolio_wts.index))]-1
fwd_return = adjclose.loc[(adjclose.index.isin(portfolio_wts.index))].shift(-1)/adjclose.loc[(adjclose.index.isin(portfolio_wts.index))]
portfolio_wts[portfolio_wts<0.01] = 0
port_ret = portfolio_wts*adjclose_return
cumulative_return_series = (port_ret.sum(axis=1)+1).cumprod()
cumulative_return = (cumulative_return.tail(1)-1)
annualized_ret = (cumulative_return_series.tail(1).values[0]**(12/len(cumulative_return_series))-1)
annualized_vol =  port_ret.sum(axis=1).std()*np.sqrt(12)

In [427]:
print("Cumulative return: " , cumulative_return)
print("Portfolio Annual returs: ",annualized_ret )
print("Portfolio Vol: ",annualized_vol)
print("Portfolio sharpe: ", annualized_ret/annualized_vol)

Cumulative return:  2023-11-01   -0.461971
dtype: float64
Portfolio Annual returs:  0.04348516838239025
Portfolio Vol:  0.0913396068122931
Portfolio sharpe:  0.476082281279732


In [385]:
portfolio_wts

,SPY,EEM,TIP,IAU,DBC,TLT,VGK,IWM,LQD,VNQ,SHY,QQQ,HYG
2012-01-03,0.025000,0.025,0.225000,0.250000,0.025,0.250,0.025,0.025,0.025,0.025,0.025000,0.02500,0.025
2012-02-01,0.025000,0.025,0.225000,0.250000,0.025,0.250,0.025,0.025,0.025,0.025,0.025000,0.02500,0.025
2012-03-01,0.025000,0.025,0.025000,0.250000,0.025,0.250,0.025,0.025,0.025,0.025,0.025000,0.22500,0.025
2012-04-02,0.025000,0.025,0.025000,0.250000,0.025,0.225,0.025,0.025,0.025,0.025,0.025000,0.25000,0.025
2012-05-01,0.025000,0.025,0.025000,0.250000,0.025,0.250,0.025,0.025,0.025,0.025,0.025000,0.22500,0.025
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-03,0.025000,0.025,0.085505,0.217710,0.025,0.025,0.025,0.025,0.025,0.025,0.205345,0.24144,0.025
2023-08-01,0.025000,0.025,0.048465,0.247207,0.025,0.025,0.025,0.025,0.025,0.025,0.204328,0.25000,0.025
2023-09-01,0.035813,0.025,0.106947,0.250000,0.025,0.025,0.025,0.025,0.025,0.025,0.132240,0.25000,0.025
2023-10-02,0.066776,0.025,0.101975,0.250000,0.025,0.025,0.025,0.025,0.025,0.025,0.106249,0.25000,0.025
